## Chargement des données

In [1]:
using CSV, DataFrames, LightGraphs

correscp = CSV.read("correspondance-code-insee-code-postal.csv", DataFrame)
data = CSV.read("migration35-commune.csv", DataFrame)

,commune,arm,dcran,achlr,agemen8,agerevq,anemc,catpc,cs1,csm
,Int64,String,String,String,String,Int64,String,Int64,Int64,String
1,35001,ZZZZZ,35001,4,55,55,4,0,5,5
2,35001,ZZZZZ,35001,4,55,50,4,0,5,5
3,35001,ZZZZZ,35001,4,55,55,4,0,5,5
4,35001,ZZZZZ,35001,4,40,50,4,0,4,4
5,35001,ZZZZZ,35001,4,40,50,4,0,4,4
6,35001,ZZZZZ,35001,6,25,35,1,0,2,2
7,35001,ZZZZZ,35001,6,55,55,2,0,6,6
8,35001,ZZZZZ,35001,6,20,20,0,0,4,4
9,35001,ZZZZZ,35001,4,55,60,4,0,7,7


Le code postal de `commune` est un `Int`, mais le `dcran` (Département et commune de résidence antérieure au 1er janvier de l'annéee précédente (arrondissement municipal pour Paris, Lyon, Marseille) est une `String`.

In [2]:
# example

data[!, "dcran"][60389]

"2A004"

C'est la Corse ! On utilise donc des `String` pour représenter les codes INSEE (et pas les code postaux).

## Construction du graphe

Chaque migration (code postal départ, code postal arrivée)

In [3]:
# maps all postcode to an int

# all postal code in string format
allcps = unique(cat(string.(data[!, "commune"]), data[!, "dcran"], dims=1))

# maps all postal code to it's position
mapping = Dict{String,Int}()

for (i,cp) in enumerate(allcps) 
   mapping[cp] = i
end

display(mapping)

Dict{String, Int64} with 2857 entries:
  "78138" => 699
  "37031" => 1153
  "91461" => 2447
  "35159" => 145
  "22321" => 1497
  "77379" => 2543
  "14061" => 1605
  "33349" => 1879
  "35266" => 248
  "31263" => 951
  "83123" => 1476
  "44068" => 1543
  "49135" => 1983
  "14648" => 1139
  "44154" => 1544
  "35285" => 265
  "97120" => 2501
  "29175" => 1827
  "44211" => 2762
  "76758" => 694
  "45282" => 1456
  "45050" => 499
  "60369" => 2687
  "56041" => 649
  "35208" => 193
  ⋮       => ⋮

In [4]:
g = SimpleDiGraph(length(allcps))

for r in eachrow(data)
    src = r["dcran"]
    dst = string(r["commune"])
    add_edge!(g, mapping[src], mapping[dst]) # ignores sel-connected nodes
end

is_connected(g)

false

Le graphe n'est pas connexe, regardons les régions isolées des autres.

In [5]:
connected_components(g)

2-element Vector{Vector{Int64}}:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  2848, 2849, 2850, 2851, 2852, 2853, 2854, 2855, 2856, 2857]
 [299]

In [6]:
allcps[299]

"35325"

Le code postal 35325 est isolé du reste, il n'est connecté qu'avec lui-même. Cela veut dire que lors du recensement de 2018, personne n'avait rejoint ou quitté La Selle-Guerchaise dans les cinq année précédentes. Retirons-le donc pour visualiser le reste du graphe.

In [12]:
using GraphPlot, Compose

partialg = deepcopy(g)
rem_vertex!(partialg, 299)
gplot(partialg, arrowlengthfrac=0) |> SVG("migrations.svg")

false

Nous voyons l'existence de gros hubs et de groupes indépendants. Regardons les zones les plus attractives. Mais avant ça, établissons la correspondance entre le code INSEE et le nom

In [8]:
codeinsee2commune = Dict{String, String}()

for r in eachrow(correscp)
    insee = r["Code INSEE"]
    commune = r["Commune"]
    codeinsee2commune[insee] = commune
end

Regardons les endroits avec les plus grand nombre de provenances différentes.

In [9]:
indegs = indegree(g)
p = sortperm(indegs, rev=true) # highest degrees first
FIRST = 10 # look at the first postal codes

cat(indegs[p][1:FIRST], map(x->codeinsee2commune[x], allcps[p][1:FIRST]), dims=2)

10×2 Matrix{Any}:
 1804  "RENNES"
  420  "SAINT-MALO"
  345  "BRUZ"
  305  "CESSON-SEVIGNE"
  216  "FOUGERES"
  203  "SAINT-JACQUES-DE-LA-LANDE"
  203  "VITRE"
  141  "DINARD"
  139  "CHANTEPIE"
  133  "BETTON"